## Notebook to generate and update scatterer dataset

In [1]:
import sys
sys.path.append('/Users/sstk4353/packages/.')

from WIPL_python_analysis.SinglePolPlots import SinglePolPlots
from WIPL_python_analysis.DualPol import DualPol
from WIPL_python_analysis.DictToSinglePolResults import OutputDictToSinglePolResultsConverter as converter
import WIPL_python_analysis.helper_functions as hf
import sys
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

sys.path.append('./')

### Processing WIPL runs for 0.5 degrees 0-25 pitch angles

In [2]:
BASE_DRIVE_PATH = '/Users/sstk4353/Library/CloudStorage/GoogleDrive-thomas.matthews@ncas.ac.uk/My Drive/WIPL/Bernard_ellipsoid_comparison/'

In [3]:
BERNARD_PATH = 'Bernard/'
ELLIPSOID_COMPARISON = 'Ellipsoid/'

In [4]:
pol_list = ['H', 'V']
size_list = ['0553', '1000']
pitch_list = list(range(0,26))
slant = 0.5
slant_name = '05'


In [5]:
pitch_result_dict = {x : {s: {y : None for y in pitch_list} for s in size_list}for x in pol_list}

In [6]:
for pol in pol_list:
    for size in size_list:
        for pitch in pitch_list:
            path = BASE_DRIVE_PATH + BERNARD_PATH + f'parameterised/dict/body_pitch_sweep/Bernard_parameterised_560_{pol}_M_{size}_body_p{pitch}_s05_dict.pkl'
            conv = converter.from_pickle(path)
            df = conv.generate_output_df()
            
            output_path = BASE_DRIVE_PATH + BERNARD_PATH + f'parameterised/csv/body_pitch_sweep/Bernard_parameterised_560_{pol}_M_{size}_body_p{pitch}_s05_df.csv'
            df.to_csv(output_path)
            pitch_result_dict[pol][size][pitch] = df

### Generate xarray dataset

In [7]:
phi = np.arange(0, 360, 2)  # 0 to 358 in steps of 2 (180 points)
names = ['Xxanth']
sizes = [9, 17]
slants = [0.5]
pitches = list(range(26))  # 0 to 25

# Define scattering patterns
scattering_patterns = ['H_H_r', 'H_H_i', 'H_V_r', 'H_V_i', 'V_H_r', 'V_H_i', 'V_V_r', 'V_V_i']

# Create the dataset with NaNs as placeholders
data_vars = {
    pattern: (['name', 'size', 'pitch','slant', 'phi'], 
              np.full((len(names), len(sizes), len(pitches), len(slants), len(phi)), np.nan))
    for pattern in scattering_patterns
}

# Create coordinates
coords = {
    'name': names,
    'size': sizes,
    'slant': slants,
    'pitch': pitches,
    'phi': phi
}

# Create the xarray Dataset
ds = xr.Dataset(data_vars=data_vars, coords=coords)

# Optional: print summary
print(ds)

<xarray.Dataset> Size: 601kB
Dimensions:  (name: 1, size: 2, pitch: 26, slant: 1, phi: 180)
Coordinates:
  * name     (name) <U6 24B 'Xxanth'
  * size     (size) int64 16B 9 17
  * slant    (slant) float64 8B 0.5
  * pitch    (pitch) int64 208B 0 1 2 3 4 5 6 7 8 ... 17 18 19 20 21 22 23 24 25
  * phi      (phi) int64 1kB 0 2 4 6 8 10 12 14 ... 346 348 350 352 354 356 358
Data variables:
    H_H_r    (name, size, pitch, slant, phi) float64 75kB nan nan ... nan nan
    H_H_i    (name, size, pitch, slant, phi) float64 75kB nan nan ... nan nan
    H_V_r    (name, size, pitch, slant, phi) float64 75kB nan nan ... nan nan
    H_V_i    (name, size, pitch, slant, phi) float64 75kB nan nan ... nan nan
    V_H_r    (name, size, pitch, slant, phi) float64 75kB nan nan ... nan nan
    V_H_i    (name, size, pitch, slant, phi) float64 75kB nan nan ... nan nan
    V_V_r    (name, size, pitch, slant, phi) float64 75kB nan nan ... nan nan
    V_V_i    (name, size, pitch, slant, phi) float64 75kB nan na

In [8]:
name = 'Xxanth'
slant = 0.5
phi = np.arange(0,360,2)  # 0 to 358 in steps of 2 (180 points)

for pol in pol_list:
    for size, length in zip(size_list, [9,17]):
        for pitch in pitch_list:
            phi_res = pd.Series(pitch_result_dict[pol][size][pitch]['Ephi'][:-1])
            theta_res = pd.Series(pitch_result_dict[pol][size][pitch]['Etheta'][:-1]) # if not try re-lable index
            
            ds[f'{pol}_H_r'].loc[dict(name=name, size=length, pitch=pitch, slant=slant)] = np.real(phi_res)
            ds[f'{pol}_H_i'].loc[dict(name=name, size=length, pitch=pitch, slant=slant)] = np.imag(phi_res)
            ds[f'{pol}_V_r'].loc[dict(name=name, size=length, pitch=pitch, slant=slant)] = np.real(theta_res)
            ds[f'{pol}_V_i'].loc[dict(name=name, size=length, pitch=pitch, slant=slant)] = np.imag(theta_res)


In [9]:
phi_res = pd.Series(pitch_result_dict['H']['1000'][2]['Ephi'][:-1])

In [10]:
phi_res

0      0.000191+0.000099j
1      0.000199+0.000099j
2      0.000212+0.000095j
3      0.000229+0.000090j
4      0.000250+0.000082j
              ...        
175    0.000213+0.000062j
176    0.000200+0.000074j
177    0.000191+0.000084j
178    0.000187+0.000092j
179    0.000187+0.000097j
Name: Ephi, Length: 180, dtype: complex128

In [15]:
ds

<xarray.Dataset> Size: 601kB
Dimensions:  (name: 1, size: 2, pitch: 26, slant: 1, phi: 180)
Coordinates:
  * name     (name) <U6 24B 'Xxanth'
  * size     (size) int64 16B 9 17
  * slant    (slant) float64 8B 0.5
  * pitch    (pitch) int64 208B 0 1 2 3 4 5 6 7 8 ... 17 18 19 20 21 22 23 24 25
  * phi      (phi) int64 1kB 0 2 4 6 8 10 12 14 ... 346 348 350 352 354 356 358
Data variables:
    H_H_r    (name, size, pitch, slant, phi) float64 75kB 6.368e-05 ... 0.000...
    H_H_i    (name, size, pitch, slant, phi) float64 75kB 9.404e-06 ... 8.227...
    H_V_r    (name, size, pitch, slant, phi) float64 75kB -2.071e-06 ... -9.3...
    H_V_i    (name, size, pitch, slant, phi) float64 75kB 4.563e-07 ... -1.29...
    V_H_r    (name, size, pitch, slant, phi) float64 75kB -2.07e-06 ... -9.38...
    V_H_i    (name, size, pitch, slant, phi) float64 75kB 4.564e-07 ... -1.29...
    V_V_r    (name, size, pitch, slant, phi) float64 75kB 5.389e-05 ... 0.000...
    V_V_i    (name, size, pitch, slant, phi) float64 75kB 1.231e-05 ... -0.00...

In [11]:
ds.to_netcdf('bioscatterer_database_v0.001.nc')

In [12]:
x = ds.sel(name='Xxanth', size=9, pitch=0, slant=0.5)

In [14]:
x['H_V_r'].to_series()

phi
0     -2.070807e-06
2     -2.772069e-06
4     -3.472224e-06
6     -4.169759e-06
8     -4.863166e-06
           ...     
350    1.399450e-06
352    7.151496e-07
354    2.451105e-08
356   -6.710151e-07
358   -1.369953e-06
Name: H_V_r, Length: 180, dtype: float64